In [9]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 9
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15


True

In [10]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("pdf_image_loader")

LangSmith 추적을 시작합니다.
[프로젝트명]
pdf_image_loader


In [11]:
!pip install -qU langchain-teddynote

## 저장할 State 정의

In [12]:
from typing import TypedDict


# GraphState 상태를 저장하는 용도로 사용합니다.
class GraphState(TypedDict):
    filepath: str  # path
    filetype: str  # pdf
    page_numbers: list[int]  # page numbers
    batch_size: int  # batch size
    split_filepaths: list[str]  # split files
    analyzed_files: list[str]  # analyzed files
    page_elements: dict[int, dict[str, list[dict]]]  # page elements
    page_metadata: dict[int, dict]  # page metadata
    page_summary: dict[int, str]  # page summary
    images: list[str]  # image paths
    images_summary: list[str]  # image summary
    tables: list[str]  # table
    tables_summary: dict[int, str]  # table summary
    texts: list[str]  # text
    texts_summary: list[str]  # text summary

## 문서를 배치 단위로 분할(10 page)

In [13]:
import os
import pymupdf
import json
import requests
from PIL import Image

In [14]:
def split_pdf(state: GraphState):
    """
    입력 PDF를 여러 개의 작은 PDF 파일로 분할합니다.

    :param state: GraphState 객체, PDF 파일 경로와 배치 크기 정보를 포함
    :return: 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체
    """
    # PDF 파일 경로와 배치 크기 추출
    filepath = state["filepath"]
    batch_size = state["batch_size"]

    # PDF 파일 열기
    input_pdf = pymupdf.open(filepath)
    num_pages = len(input_pdf)
    print(f"총 페이지 수: {num_pages}")

    ret = []
    # PDF 분할 작업 시작
    for start_page in range(0, num_pages, batch_size):
        # 배치의 마지막 페이지 계산 (전체 페이지 수를 초과하지 않도록)
        end_page = min(start_page + batch_size, num_pages) - 1

        # 분할된 PDF 파일명 생성
        input_file_basename = os.path.splitext(filepath)[0]
        output_file = f"{input_file_basename}_{start_page:04d}_{end_page:04d}.pdf"
        print(f"분할 PDF 생성: {output_file}")

        # 새로운 PDF 파일 생성 및 페이지 삽입
        with pymupdf.open() as output_pdf:
            output_pdf.insert_pdf(input_pdf, from_page=start_page, to_page=end_page)
            output_pdf.save(output_file)
            ret.append(output_file)

    # 원본 PDF 파일 닫기
    input_pdf.close()

    # 분할된 PDF 파일 경로 목록을 포함한 GraphState 객체 반환
    return GraphState(split_filepaths=ret)

## state 등록

In [15]:
state = GraphState(filepath="data/20241121_company_54055000.pdf", batch_size=1)
state_out = split_pdf(state)
state.update(state_out)
state

총 페이지 수: 5
분할 PDF 생성: data/20241121_company_54055000_0000_0000.pdf
분할 PDF 생성: data/20241121_company_54055000_0001_0001.pdf
분할 PDF 생성: data/20241121_company_54055000_0002_0002.pdf
분할 PDF 생성: data/20241121_company_54055000_0003_0003.pdf
분할 PDF 생성: data/20241121_company_54055000_0004_0004.pdf


{'filepath': 'data/20241121_company_54055000.pdf',
 'batch_size': 1,
 'split_filepaths': ['data/20241121_company_54055000_0000_0000.pdf',
  'data/20241121_company_54055000_0001_0001.pdf',
  'data/20241121_company_54055000_0002_0002.pdf',
  'data/20241121_company_54055000_0003_0003.pdf',
  'data/20241121_company_54055000_0004_0004.pdf']}

## Layout Analyzer를 사용하여 문서를 element 단위로 분할

In [16]:
class LayoutAnalyzer:
    def __init__(self, api_key):
        """
        LayoutAnalyzer 클래스의 생성자

        :param api_key: Upstage API 인증을 위한 API 키
        """
        self.api_key = api_key

    def _upstage_layout_analysis(self, input_file):
        """
        Upstage의 레이아웃 분석 API를 호출하여 문서 분석을 수행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        # API 요청 헤더 설정
        headers = {"Authorization": f"Bearer {self.api_key}"}

        # API 요청 데이터 설정 (OCR 비활성화)
        data = {"ocr": True}

        # 분석할 PDF 파일 열기
        files = {"document": open(input_file, "rb")}

        # API 요청 보내기
        response = requests.post(
            "https://api.upstage.ai/v1/document-ai/layout-analysis",
            headers=headers,
            data=data,
            files=files,
        )

        # API 응답 처리 및 결과 저장
        if response.status_code == 200:
            # 분석 결과를 저장할 JSON 파일 경로 생성
            output_file = os.path.splitext(input_file)[0] + ".json"

            # 분석 결과를 JSON 파일로 저장
            with open(output_file, "w") as f:
                json.dump(response.json(), f, ensure_ascii=False)

            return output_file
        else:
            # API 요청이 실패한 경우 예외 발생
            raise ValueError(f"API 요청 실패. 상태 코드: {response.status_code}")

    def execute(self, input_file):
        """
        주어진 입력 파일에 대해 레이아웃 분석을 실행합니다.

        :param input_file: 분석할 PDF 파일의 경로
        :return: 분석 결과가 저장된 JSON 파일의 경로
        """
        return self._upstage_layout_analysis(input_file)

In [17]:
def analyze_layout(state: GraphState):
    # 분할된 PDF 파일 목록을 가져옵니다.
    split_files = state["split_filepaths"]

    # LayoutAnalyzer 객체를 생성합니다. API 키는 환경 변수에서 가져옵니다.
    analyzer = LayoutAnalyzer(os.environ.get("UPSTAGE_API_KEY"))

    # 분석된 파일들의 경로를 저장할 리스트를 초기화합니다.
    analyzed_files = []

    # 각 분할된 PDF 파일에 대해 레이아웃 분석을 수행합니다.
    for file in split_files:
        # 레이아웃 분석을 실행하고 결과 파일 경로를 리스트에 추가합니다.
        analyzed_files.append(analyzer.execute(file))

    # 분석된 파일 경로들을 정렬하여 새로운 GraphState 객체를 생성하고 반환합니다.
    # 정렬은 파일들의 순서를 유지하기 위해 수행됩니다.
    return GraphState(analyzed_files=sorted(analyzed_files))

In [18]:
state_out = analyze_layout(state)
state.update(state_out)
state

{'filepath': 'data/20241121_company_54055000.pdf',
 'batch_size': 1,
 'split_filepaths': ['data/20241121_company_54055000_0000_0000.pdf',
  'data/20241121_company_54055000_0001_0001.pdf',
  'data/20241121_company_54055000_0002_0002.pdf',
  'data/20241121_company_54055000_0003_0003.pdf',
  'data/20241121_company_54055000_0004_0004.pdf'],
 'analyzed_files': ['data/20241121_company_54055000_0000_0000.json',
  'data/20241121_company_54055000_0001_0001.json',
  'data/20241121_company_54055000_0002_0002.json',
  'data/20241121_company_54055000_0003_0003.json',
  'data/20241121_company_54055000_0004_0004.json']}

## 페이지 메타데이터 추출

In [19]:
import re


def extract_start_end_page(filename):
    """
    파일 이름에서 시작 페이지와 끝 페이지 번호를 추출하는 함수입니다.

    :param filename: 분석할 파일의 이름
    :return: 시작 페이지 번호와 끝 페이지 번호를 튜플로 반환
    """
    # 파일 경로에서 파일 이름만 추출
    file_name = os.path.basename(filename)
    # 파일 이름을 '_' 기준으로 분리
    file_name_parts = file_name.split("_")

    if len(file_name_parts) >= 3:
        # 파일 이름의 뒤에서 두 번째 부분에서 숫자를 추출하여 시작 페이지로 설정
        start_page = int(re.findall(r"(\d+)", file_name_parts[-2])[0])
        # 파일 이름의 마지막 부분에서 숫자를 추출하여 끝 페이지로 설정
        end_page = int(re.findall(r"(\d+)", file_name_parts[-1])[0])
    else:
        # 파일 이름 형식이 예상과 다를 경우 기본값 설정
        start_page, end_page = 0, 0

    return start_page, end_page


def extract_page_metadata(state: GraphState):
    """
    분석된 JSON 파일들에서 페이지 메타데이터를 추출하는 함수입니다.

    :param state: 현재의 GraphState 객체
    :return: 페이지 메타데이터가 추가된 새로운 GraphState 객체
    """
    # 분석된 JSON 파일 목록 가져오기
    json_files = state["analyzed_files"]

    # 페이지 메타데이터를 저장할 딕셔너리 초기화
    page_metadata = dict()

    for json_file in json_files:
        # JSON 파일 열기 및 데이터 로드
        with open(json_file, "r") as f:
            data = json.load(f)

        # 파일명에서 시작 페이지 번호 추출
        start_page, _ = extract_start_end_page(json_file)

        # JSON 데이터에서 각 페이지의 메타데이터 추출
        for element in data["metadata"]["pages"]:
            # 원본 페이지 번호
            original_page = int(element["page"])
            # 상대적 페이지 번호 계산 (전체 문서 기준)
            relative_page = start_page + original_page - 1

            # 페이지 크기 정보 추출
            metadata = {
                "size": [
                    int(element["width"]),
                    int(element["height"]),
                ],
            }
            # 상대적 페이지 번호를 키로 하여 메타데이터 저장
            page_metadata[relative_page] = metadata

    # 추출된 페이지 메타데이터로 새로운 GraphState 객체 생성 및 반환
    return GraphState(page_metadata=page_metadata)

페이지별 사이즈를 추출합니다.

In [20]:
state_out = extract_page_metadata(state)
state.update(state_out)
state["page_metadata"]

{0: {'size': [1241, 1754]},
 1: {'size': [1241, 1754]},
 2: {'size': [1241, 1754]},
 3: {'size': [1241, 1754]},
 4: {'size': [1241, 1754]}}

## 페이지별 HTML Element 추출

In [21]:
def extract_page_elements(state: GraphState):
    # 분석된 JSON 파일 목록을 가져옵니다.
    json_files = state["analyzed_files"]

    # 페이지별 요소를 저장할 딕셔너리를 초기화합니다.
    page_elements = dict()

    # 전체 문서에서 고유한 요소 ID를 부여하기 위한 카운터입니다.
    element_id = 0

    # 각 JSON 파일을 순회하며 처리합니다.
    for json_file in json_files:
        # 파일명에서 시작 페이지 번호를 추출합니다.
        start_page, _ = extract_start_end_page(json_file)

        # JSON 파일을 열어 데이터를 로드합니다.
        with open(json_file, "r") as f:
            data = json.load(f)

        # JSON 데이터의 각 요소를 처리합니다.
        for element in data["elements"]:
            # 원본 페이지 번호를 정수로 변환합니다.
            original_page = int(element["page"])
            # 전체 문서 기준의 상대적 페이지 번호를 계산합니다.
            relative_page = start_page + original_page - 1

            # 해당 페이지의 요소 리스트가 없으면 새로 생성합니다.
            if relative_page not in page_elements:
                page_elements[relative_page] = []

            # 요소에 고유 ID를 부여합니다.
            element["id"] = element_id
            element_id += 1

            # 요소의 페이지 번호를 상대적 페이지 번호로 업데이트합니다.
            element["page"] = relative_page
            # 요소를 해당 페이지의 리스트에 추가합니다.
            page_elements[relative_page].append(element)

    # 추출된 페이지별 요소 정보로 새로운 GraphState 객체를 생성하여 반환합니다.
    return GraphState(page_elements=page_elements)

In [22]:
state_out = extract_page_elements(state)
state.update(state_out)
state["page_elements"][1]

[{'bounding_box': [{'x': 1001, 'y': 52},
   {'x': 1151, 'y': 52},
   {'x': 1151, 'y': 77},
   {'x': 1001, 'y': 77}],
  'category': 'paragraph',
  'html': "<p id='0' data-category='paragraph' style='font-size:22px'>한국단자 (025540)</p>",
  'id': 10,
  'page': 1,
  'text': '한국단자 (025540)'},
 {'bounding_box': [{'x': 85, 'y': 140},
   {'x': 316, 'y': 140},
   {'x': 316, 'y': 163},
   {'x': 85, 'y': 163}],
  'category': 'caption',
  'html': "<caption id='1' style='font-size:18px'>도표 1. 한국단자의 실적 추이</caption>",
  'id': 11,
  'page': 1,
  'text': '도표 1. 한국단자의 실적 추이'},
 {'bounding_box': [{'x': 1055, 'y': 142},
   {'x': 1154, 'y': 142},
   {'x': 1154, 'y': 164},
   {'x': 1055, 'y': 164}],
  'category': 'paragraph',
  'html': "<br><p id='2' data-category='paragraph' style='font-size:16px'>(단위: 십억원, %)</p>",
  'id': 12,
  'page': 1,
  'text': '(단위: 십억원, %)'},
 {'bounding_box': [{'x': 84, 'y': 160},
   {'x': 1161, 'y': 160},
   {'x': 1161, 'y': 1066},
   {'x': 84, 'y': 1066}],
  'category': 'table',
 

추출된 페이지를 확인

In [23]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3, 4])

## 각 페이지의 태그를 분해합니다.
- images, tables, text를 분해합니다.

In [24]:
def extract_tag_elements_per_page(state: GraphState):
    # GraphState 객체에서 페이지 요소들을 가져옵니다.
    page_elements = state["page_elements"]

    # 파싱된 페이지 요소들을 저장할 새로운 딕셔너리를 생성합니다.
    parsed_page_elements = dict()

    # 각 페이지와 해당 페이지의 요소들을 순회합니다.
    for key, page_element in page_elements.items():
        # 이미지, 테이블, 텍스트 요소들을 저장할 리스트를 초기화합니다.
        image_elements = []
        table_elements = []
        text_elements = []

        # 페이지의 각 요소를 순회하며 카테고리별로 분류합니다.
        for element in page_element:
            if element["category"] == "chart":
                # 이미지 요소인 경우 image_elements 리스트에 추가합니다.
                image_elements.append(element)
            elif element["category"] == "table":
                # 테이블 요소인 경우 table_elements 리스트에 추가합니다.
                table_elements.append(element)
            else:
                # 그 외의 요소는 모두 텍스트 요소로 간주하여 text_elements 리스트에 추가합니다.
                text_elements.append(element)

        # 분류된 요소들을 페이지 키와 함께 새로운 딕셔너리에 저장합니다.
        parsed_page_elements[key] = {
            "image_elements": image_elements,
            "table_elements": table_elements,
            "text_elements": text_elements,
            "elements": page_element,  # 원본 페이지 요소도 함께 저장합니다.
        }

    # 파싱된 페이지 요소들을 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(page_elements=parsed_page_elements)

In [25]:
state_out = extract_tag_elements_per_page(state)
state.update(state_out)

In [26]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3, 4])

In [27]:
state["page_elements"][1].keys()

dict_keys(['image_elements', 'table_elements', 'text_elements', 'elements'])

In [28]:
state["page_elements"][1]["image_elements"]

[{'bounding_box': [{'x': 97, 'y': 1205},
   {'x': 599, 'y': 1205},
   {'x': 599, 'y': 1541},
   {'x': 97, 'y': 1541}],
  'category': 'chart',
  'html': '<figure data-category=\'chart\'><img id=\'6\' style=\'font-size:14px\' alt="(십억원)\n■ 매출액 순이익\n100\n89.3\n85.3\n90\n80\n70\n60 53.8\n50\n40\n28.6\n30 20.4\n14.0\n20\n4.9 5.9 7.1 7.2\n10 0.5 1.4 1.0 1.7 2.5\n0\n(10) 0.0\n2022 2023 2024" data-coord="top-left:(97,1205); bottom-right:(599,1541)" /></figure>',
  'id': 16,
  'page': 1,
  'text': '(십억원)\n■ 매출액 순이익\n100\n89.3\n85.3\n90\n80\n70\n60 53.8\n50\n40\n28.6\n30 20.4\n14.0\n20\n4.9 5.9 7.1 7.2\n10 0.5 1.4 1.0 1.7 2.5\n0\n(10) 0.0\n2022 2023 2024'},
 {'bounding_box': [{'x': 648, 'y': 1206},
   {'x': 1142, 'y': 1206},
   {'x': 1142, 'y': 1536},
   {'x': 648, 'y': 1536}],
  'category': 'chart',
  'html': '<figure data-category=\'chart\'><img id=\'9\' style=\'font-size:14px\' alt="(십억원)\n매출액 순이익\n30\n26.4\n23.7\n25\n20\n15.8\n15\n8.3\n10\n6.5\n3.2 2.9\n5\n1.5 0.9 0.8\n1.6\n1.0 0.8 0.6\n0\n-

In [29]:
state["page_elements"][4]["table_elements"]

[{'bounding_box': [{'x': 665, 'y': 1567},
   {'x': 1120, 'y': 1567},
   {'x': 1120, 'y': 1608},
   {'x': 665, 'y': 1608}],
  'category': 'table',
  'html': "<br><table id='16' style='font-size:16px'><tr><td>투자등급</td><td>BUY(매수)</td><td>Neutral(중립)</td><td>Reduce(매도)</td><td>합계</td></tr><tr><td>금융투자상품의 비율</td><td>93.75%</td><td>5.80%</td><td>0.45%</td><td>100%</td></tr></table>",
  'id': 70,
  'page': 4,
  'text': '투자등급 BUY(매수) Neutral(중립) Reduce(매도) 합계\n 금융투자상품의 비율 93.75% 5.80% 0.45% 100%'}]

In [30]:
state["page_elements"][2]["text_elements"]

[{'bounding_box': [{'x': 93, 'y': 140},
   {'x': 430, 'y': 140},
   {'x': 430, 'y': 163},
   {'x': 93, 'y': 163}],
  'category': 'caption',
  'html': "<caption id='0' style='font-size:18px'>도표 4. 한국단자의 자동차용 ASP 상승률 추이</caption>",
  'id': 22,
  'page': 2,
  'text': '도표 4. 한국단자의 자동차용 ASP 상승률 추이'},
 {'bounding_box': [{'x': 92, 'y': 528},
   {'x': 237, 'y': 528},
   {'x': 237, 'y': 550},
   {'x': 92, 'y': 550}],
  'category': 'paragraph',
  'html': "<br><p id='2' data-category='paragraph' style='font-size:18px'>자료: 한국단자, 하나증권</p>",
  'id': 24,
  'page': 2,
  'text': '자료: 한국단자, 하나증권'},
 {'bounding_box': [{'x': 84, 'y': 609},
   {'x': 405, 'y': 609},
   {'x': 405, 'y': 633},
   {'x': 84, 'y': 633}],
  'category': 'caption',
  'html': "<caption id='3' style='font-size:18px'>도표 6. 국제 구리 가격 추이 (LME Copper)</caption>",
  'id': 25,
  'page': 2,
  'text': '도표 6. 국제 구리 가격 추이 (LME Copper)'},
 {'bounding_box': [{'x': 1000, 'y': 51},
   {'x': 1152, 'y': 51},
   {'x': 1152, 'y': 78},
   {'x': 1000, 'y'

## 페이지 번호를 추출합니다.

In [31]:
def page_numbers(state: GraphState):
    return GraphState(page_numbers=list(state["page_elements"].keys()))


state_out = page_numbers(state)
state.update(state_out)
state["page_numbers"]

[0, 1, 2, 3, 4]

## 이미지 추출합니다.

In [32]:
class ImageCropper:
    @staticmethod
    def pdf_to_image(pdf_file, page_num, dpi=300):
        """
        PDF 파일의 특정 페이지를 이미지로 변환하는 메서드

        :param page_num: 변환할 페이지 번호 (1부터 시작)
        :param dpi: 이미지 해상도 (기본값: 300)
        :return: 변환된 이미지 객체
        """
        with pymupdf.open(pdf_file) as doc:
            page = doc[page_num].get_pixmap(dpi=dpi)
            target_page_size = [page.width, page.height]
            page_img = Image.frombytes("RGB", target_page_size, page.samples)
        return page_img

    @staticmethod
    def normalize_coordinates(coordinates, output_page_size):
        """
        좌표를 정규화하는 정적 메서드

        :param coordinates: 원본 좌표 리스트
        :param output_page_size: 출력 페이지 크기 [너비, 높이]
        :return: 정규화된 좌표 (x1, y1, x2, y2)
        """
        x_values = [coord["x"] for coord in coordinates]
        y_values = [coord["y"] for coord in coordinates]
        x1, y1, x2, y2 = min(x_values), min(y_values), max(x_values), max(y_values)

        return (
            x1 / output_page_size[0],
            y1 / output_page_size[1],
            x2 / output_page_size[0],
            y2 / output_page_size[1],
        )

    @staticmethod
    def crop_image(img, coordinates, output_file):
        """
        이미지를 주어진 좌표에 따라 자르고 저장하는 정적 메서드

        :param img: 원본 이미지 객체
        :param coordinates: 정규화된 좌표 (x1, y1, x2, y2)
        :param output_file: 저장할 파일 경로
        """
        img_width, img_height = img.size
        x1, y1, x2, y2 = [
            int(coord * dim)
            for coord, dim in zip(coordinates, [img_width, img_height] * 2)
        ]
        cropped_img = img.crop((x1, y1, x2, y2))
        cropped_img.save(output_file)

In [33]:
sample_table = state["page_elements"][0]["table_elements"][0]
sample_table

{'bounding_box': [{'x': 67, 'y': 74},
  {'x': 358, 'y': 74},
  {'x': 358, 'y': 1682},
  {'x': 67, 'y': 1682}],
 'category': 'table',
 'html': '<br><table id=\'1\' style=\'font-size:14px\'><tr><td colspan="5">Not Rated</td></tr><tr><td colspan="3">목표주가(12M)</td><td colspan="2">Not Rated</td></tr><tr><td colspan="3">현재주가(11.20)</td><td colspan="2">70,500원</td></tr><tr><td colspan="5">Key Data</td></tr><tr><td colspan="3">KOSPI 지수 (pt)</td><td colspan="2">2,482,29</td></tr><tr><td colspan="3">52주 최고/최저(원)</td><td colspan="2">78,200/55,200</td></tr><tr><td colspan="3">시가총액(십억원)</td><td colspan="2">734.3</td></tr><tr><td colspan="3">시가총액비중(%)</td><td colspan="2">0.04</td></tr><tr><td colspan="3">발행주식수(천주)</td><td colspan="2">10,415.0</td></tr><tr><td colspan="3">60일 평균 거래량(천주)</td><td colspan="2">37.5</td></tr><tr><td colspan="3">60일 평균 거래대금(십억원)</td><td colspan="2">2.8</td></tr><tr><td colspan="3">외국인지분율(%) 주요주주 지분율(%)</td><td colspan="2">28,18</td></tr><tr><td colspan="3">이창원 외 10 인</td><

In [34]:
sample_table = state["page_elements"][1]["table_elements"][0]
sample_table

{'bounding_box': [{'x': 84, 'y': 160},
  {'x': 1161, 'y': 160},
  {'x': 1161, 'y': 1066},
  {'x': 84, 'y': 1066}],
 'category': 'table',
 'html': "<br><table id='3' style='font-size:16px'><tr><td>구분</td><td>1Q22 9.7</td><td>2Q22</td><td>3Q22</td><td>4Q22</td><td>1Q23</td><td>2Q23</td><td>3Q23</td><td>4Q23</td><td>1Q24</td><td>2Q24</td><td>3Q24</td><td>2020</td><td>2021 7.9 9.0</td><td>2022</td><td>2023</td></tr><tr><td>매출액</td><td>272.3 7.0</td><td>286.8</td><td>290.8 7.0</td><td>318.2</td><td>315.9</td><td>333.0 4.2</td><td>318.5</td><td>329.5</td><td>352.1</td><td>391.4</td><td>378.0</td><td>802.5</td><td>962.2</td><td>1,168.1 8.4</td><td>1,296.9</td></tr><tr><td>(제품별)</td><td></td><td></td><td></td><td></td><td></td><td></td><td>8.9</td><td></td><td></td><td></td><td></td><td>7.2</td><td></td><td></td><td></td></tr><tr><td>차량용</td><td>234.8 2.6</td><td>247.4</td><td>274.1</td><td>311.2</td><td>295.8 8.7</td><td>323.8</td><td>328.3</td><td>327.1</td><td>358.7</td><td>497.0</td><td>35

## 이미지 추출

In [35]:
def crop_image(state: GraphState):
    """
    PDF 파일에서 이미지를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["image_elements"]:
            if element["category"] == "chart":
                # 이미지 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        images=cropped_images
    )  # 크롭된 이미지 정보를 포함한 GraphState 반환


def crop_table(state: GraphState):
    """
    PDF 파일에서 표를 추출하고 크롭하는 함수

    :param state: GraphState 객체
    :return: 크롭된 표 이미지 정보가 포함된 GraphState 객체
    """
    pdf_file = state["filepath"]  # PDF 파일 경로
    page_numbers = state["page_numbers"]  # 처리할 페이지 번호 목록
    output_folder = os.path.splitext(pdf_file)[0]  # 출력 폴더 경로 설정
    os.makedirs(output_folder, exist_ok=True)  # 출력 폴더 생성

    cropped_images = dict()  # 크롭된 표 이미지 정보를 저장할 딕셔너리
    for page_num in page_numbers:
        pdf_image = ImageCropper.pdf_to_image(
            pdf_file, page_num
        )  # PDF 페이지를 이미지로 변환
        for element in state["page_elements"][page_num]["table_elements"]:
            if element["category"] == "table":
                # 표 요소의 좌표를 정규화
                normalized_coordinates = ImageCropper.normalize_coordinates(
                    element["bounding_box"], state["page_metadata"][page_num]["size"]
                )

                # 크롭된 표 이미지 저장 경로 설정
                output_file = os.path.join(output_folder, f"{element['id']}.png")
                # 표 이미지 크롭 및 저장
                ImageCropper.crop_image(pdf_image, normalized_coordinates, output_file)
                cropped_images[element["id"]] = output_file
                print(f"page:{page_num}, id:{element['id']}, path: {output_file}")
    return GraphState(
        tables=cropped_images
    )  # 크롭된 표 이미지 정보를 포함한 GraphState 반환

In [36]:
state["page_elements"].keys()

dict_keys([0, 1, 2, 3, 4])

In [37]:
state_out = crop_image(state)
state.update(state_out)
state["images"]
# state.update(state6)

page:1, id:16, path: data/20241121_company_54055000\16.png
page:1, id:19, path: data/20241121_company_54055000\19.png
page:2, id:23, path: data/20241121_company_54055000\23.png
page:2, id:28, path: data/20241121_company_54055000\28.png
page:2, id:30, path: data/20241121_company_54055000\30.png
page:2, id:33, path: data/20241121_company_54055000\33.png
page:2, id:36, path: data/20241121_company_54055000\36.png
page:4, id:57, path: data/20241121_company_54055000\57.png


{16: 'data/20241121_company_54055000\\16.png',
 19: 'data/20241121_company_54055000\\19.png',
 23: 'data/20241121_company_54055000\\23.png',
 28: 'data/20241121_company_54055000\\28.png',
 30: 'data/20241121_company_54055000\\30.png',
 33: 'data/20241121_company_54055000\\33.png',
 36: 'data/20241121_company_54055000\\36.png',
 57: 'data/20241121_company_54055000\\57.png'}

In [38]:
state_out = crop_table(state)
state.update(state_out)
state["tables"]

page:0, id:1, path: data/20241121_company_54055000\1.png
page:1, id:13, path: data/20241121_company_54055000\13.png
page:3, id:43, path: data/20241121_company_54055000\43.png
page:3, id:46, path: data/20241121_company_54055000\46.png
page:3, id:50, path: data/20241121_company_54055000\50.png
page:3, id:52, path: data/20241121_company_54055000\52.png
page:4, id:70, path: data/20241121_company_54055000\70.png


{1: 'data/20241121_company_54055000\\1.png',
 13: 'data/20241121_company_54055000\\13.png',
 43: 'data/20241121_company_54055000\\43.png',
 46: 'data/20241121_company_54055000\\46.png',
 50: 'data/20241121_company_54055000\\50.png',
 52: 'data/20241121_company_54055000\\52.png',
 70: 'data/20241121_company_54055000\\70.png'}

## 페이지 텍스트 추출

In [39]:
def extract_page_text(state: GraphState):
    # 상태 객체에서 페이지 번호 목록을 가져옵니다.
    page_numbers = state["page_numbers"]

    # 추출된 텍스트를 저장할 딕셔너리를 초기화합니다.
    extracted_texts = dict()

    # 각 페이지 번호에 대해 반복합니다.
    for page_num in page_numbers:
        # 현재 페이지의 텍스트를 저장할 빈 문자열을 초기화합니다.
        extracted_texts[page_num] = ""

        # 현재 페이지의 모든 텍스트 요소에 대해 반복합니다.
        for element in state["page_elements"][page_num]["text_elements"]:
            # 각 텍스트 요소의 내용을 현재 페이지의 텍스트에 추가합니다.
            extracted_texts[page_num] += element["text"]

    # 추출된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(texts=extracted_texts)

In [40]:
state_out = extract_page_text(state)
state.update(state_out)
state["texts"]

{0: '2024년 11월 21일 I 기업분석한국단자 (025540)또 다시 예상보다 좋았어요3Q24 Review: 영업이익률 12.8% 기록한국단자의 3분기 매출액/영업이익은 19%/48% (YoY) 증가한 3,780억원/485억원(영업이\n익률 12.8%, +2.5%p (YoY))으로 2분기에 이어 또 한번 서프라이즈를 기록했다(영업이익\n기준 컨센서스 대비 +30%). 한국(본사/케이이티솔루션/케이티네트워크) 매출액이 6%\n(YoY) 증가했고, 폴란드 매출액은 17% (YoY) 감소했지만 미국 매출액이 317% (YoY) 급\n증했다. 미국/멕시코 법인의 합산 매출액 비중은 21%로 15%p (YoY) 상승했다. 믹스 효\n과로 차량용 커넥터의 평균가격은 3분기 누적 5% (YoY) 상승했다. 구리 등 비철원재료와\n수지원재료(PBT 등)의 투입원가는 누적 기준 각각 +2.3%/-4.4% (YoY) 변동했다. 주요 공\n장의 가동률이 5%p (YoY) 상승하면서 고정비 레버리지 효과가 발생하여 매출원가율이 -\n2.6%p (YoY), +1.2%p (QoQ) 변동했다. 결과로 매출총이익률이 20.8%를 기록했고, 영업\n이익률은 +2.5%p (YoY), -1.7%p (QoQ) 변동한 12.8%로 2분기 14.5%에 이어 2020년\n이후 두 번째로 높은 분기 수익성을 보였다. 3분기가 계절적 비수기임을 고려할 때 3분기\n기준으로는 최고치이다. 영업외 외환손실로 세전이익/지배주주순이익은 영업이익 증가율\n보다 낮은 16%/26% (YoY) 증가한 393억원/338억원을 기록했다.미국 법인의 3분기 누적 매출액이 전년 연간 매출액의 3.3배까지 증가북미 판매/생산을 각각 담당하는 미국/멕시코 법인은 전기차용 ICB(Inter-Connect Board)\n위주로 급성장을 이어가고 있다. 미국/멕시코는 2022년 첫 매출액이 발생한 후 2023년\n689억원/195원을 기록했고, 2024년에는 분기 매출액이 전년 연간 수준을 기록 중이다. 1\n분기 538억원

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import (
    create_stuff_documents_chain,
)
from langchain_core.documents import Document

# 요약을 위한 프롬프트 템플릿을 정의합니다.
prompt = PromptTemplate.from_template(
    """아래의 규칙에 따라 차트와 표를 요약해줘
    
규칙:
1. 보기 쉽게 넘버링으로 요약해줘.
2. 한국어로 작성해줘.
3. 기계적 용어는 번역하지 말고, 원문 그대로 보여줘.
4. 불필요한 정보는 포함하지 말아줘.
5. 중요한 실제 값과 수치를 포함해서 요약해줘.

CONTEXT:
{context}

SUMMARY:"
"""
)

# ChatOpenAI 모델의 또 다른 인스턴스를 생성합니다. (이전 인스턴스와 동일한 설정)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

# 문서 요약을 위한 체인을 생성합니다.
# 이 체인은 여러 문서를 입력받아 하나의 요약된 텍스트로 결합합니다.
text_summary_chain = create_stuff_documents_chain(llm, prompt)

C:\Users\user\AppData\Local\Temp\ipykernel_6752\4093944364.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [42]:
def create_text_summary(state: GraphState):
    # state에서 텍스트 데이터를 가져옵니다.
    texts = state["texts"]

    # 요약된 텍스트를 저장할 딕셔너리를 초기화합니다.
    text_summary = dict()

    # texts.items()를 페이지 번호(키)를 기준으로 오름차순 정렬합니다.
    sorted_texts = sorted(texts.items(), key=lambda x: x[0])

    # 각 페이지의 텍스트를 Document 객체로 변환하여 입력 리스트를 생성합니다.
    inputs = [
        {"context": [Document(page_content=text)]} for page_num, text in sorted_texts
    ]

    # text_summary_chain을 사용하여 일괄 처리로 요약을 생성합니다.
    summaries = text_summary_chain.batch(inputs)

    # 생성된 요약을 페이지 번호와 함께 딕셔너리에 저장합니다.
    for page_num, summary in enumerate(summaries):
        text_summary[page_num] = summary

    # 요약된 텍스트를 포함한 새로운 GraphState 객체를 반환합니다.
    return GraphState(text_summary=text_summary)


# create_text_summary 함수를 호출하여 텍스트 요약을 생성합니다.
state_out = create_text_summary(state)

# 생성된 요약을 기존 state에 업데이트합니다.
state.update(state_out)

# 요약된 텍스트를 출력합니다.
state_out["text_summary"]

{0: '1. **3분기 실적**: 한국단자는 3Q24에서 매출액 3,780억원, 영업이익 485억원을 기록함. 영업이익률은 12.8%로, 전년 대비 +2.5%p 증가.\n\n2. **매출 증가율**: 매출액은 전년 대비 19% 증가, 영업이익은 48% 증가. 영업이익 기준 컨센서스 대비 +30% 서프라이즈.\n\n3. **지역별 매출**:\n   - 한국: 매출액 6% 증가 (YoY)\n   - 폴란드: 매출액 17% 감소 (YoY)\n   - 미국: 매출액 317% 급증 (YoY)\n\n4. **미국/멕시코 법인**: 합산 매출액 비중 21%로, 전년 대비 15%p 상승. 3분기 누적 매출액은 2,283억원으로, 2023년 연간 매출액의 3.3배.\n\n5. **원가 및 수익성**: \n   - 차량용 커넥터 평균가격 5% 상승 (YoY)\n   - 구리 등 비철원재료 원가는 +2.3% 변동, 수지원재료는 -4.4% 변동.\n   - 매출원가율은 -2.6%p (YoY) 감소, 매출총이익률 20.8% 기록.\n\n6. **순이익**: 3분기 세전이익 393억원, 지배주주순이익 338억원으로 각각 16% 및 26% 증가 (YoY).\n\n7. **기대 성장**: 기수주 물량, 신규 고객 발굴, 멕시코 2공장 증설로 높은 성장 기대.\n\n8. **주가 평가**: 현재 P/E 5배대로 저평가 상태, 연간 지배주주순이익 1,400억원 이상 예상.',
 1: '1. **한국단자 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **미국 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]\n\n3. **멕시코 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]  \n\n*각 항목의 구체적인 수치는

## Image, Table 요약을 위한 데이터 배치 생성

In [43]:
state["page_elements"][1]["image_elements"][0]

{'bounding_box': [{'x': 97, 'y': 1205},
  {'x': 599, 'y': 1205},
  {'x': 599, 'y': 1541},
  {'x': 97, 'y': 1541}],
 'category': 'chart',
 'html': '<figure data-category=\'chart\'><img id=\'6\' style=\'font-size:14px\' alt="(십억원)\n■ 매출액 순이익\n100\n89.3\n85.3\n90\n80\n70\n60 53.8\n50\n40\n28.6\n30 20.4\n14.0\n20\n4.9 5.9 7.1 7.2\n10 0.5 1.4 1.0 1.7 2.5\n0\n(10) 0.0\n2022 2023 2024" data-coord="top-left:(97,1205); bottom-right:(599,1541)" /></figure>',
 'id': 16,
 'page': 1,
 'text': '(십억원)\n■ 매출액 순이익\n100\n89.3\n85.3\n90\n80\n70\n60 53.8\n50\n40\n28.6\n30 20.4\n14.0\n20\n4.9 5.9 7.1 7.2\n10 0.5 1.4 1.0 1.7 2.5\n0\n(10) 0.0\n2022 2023 2024'}

In [44]:
print(state["text_summary"][1])

1. **한국단자 실적 추이**  
   - 단위: 십억원, %  
   - 주요 실적: 매출, 영업이익, 순이익 등

2. **미국 법인 매출액/순이익 추이**  
   - 매출액: [구체적인 수치]  
   - 순이익: [구체적인 수치]  
   - 연도별 변화: [구체적인 변화]

3. **멕시코 법인 매출액/순이익 추이**  
   - 매출액: [구체적인 수치]  
   - 순이익: [구체적인 수치]  
   - 연도별 변화: [구체적인 변화]  

*각 항목의 구체적인 수치는 원문에서 확인 필요.*


In [45]:
def create_image_summary_data_batches(state: GraphState):
    # 이미지 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 이미지 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["image_elements"]:
            # 이미지 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 이미지 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "image": state["images"][image_id],  # 이미지 파일 경로
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 이미지 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(image_summary_data_batches=data_batches)


def create_table_summary_data_batches(state: GraphState):
    # 테이블 요약을 위한 데이터 배치를 생성하는 함수
    data_batches = []

    # 페이지 번호를 오름차순으로 정렬
    page_numbers = sorted(list(state["page_elements"].keys()))

    for page_num in page_numbers:
        # 각 페이지의 요약된 텍스트를 가져옴
        text = state["text_summary"][page_num]
        # 해당 페이지의 모든 테이블 요소에 대해 반복
        for image_element in state["page_elements"][page_num]["table_elements"]:
            # 테이블 ID를 정수로 변환
            image_id = int(image_element["id"])

            # 데이터 배치에 테이블 정보, 관련 텍스트, 페이지 번호, ID를 추가
            data_batches.append(
                {
                    "table": state["tables"][image_id],  # 테이블 데이터
                    "text": text,  # 관련 텍스트 요약
                    "page": page_num,  # 페이지 번호
                    "id": image_id,  # 테이블 ID
                }
            )
    # 생성된 데이터 배치를 GraphState 객체에 담아 반환
    return GraphState(table_summary_data_batches=data_batches)

In [46]:
state_out = create_image_summary_data_batches(state)
state.update(state_out)
state_out

{'image_summary_data_batches': [{'image': 'data/20241121_company_54055000\\16.png',
   'text': '1. **한국단자 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **미국 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]\n\n3. **멕시코 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]  \n\n*각 항목의 구체적인 수치는 원문에서 확인 필요.*',
   'page': 1,
   'id': 16},
  {'image': 'data/20241121_company_54055000\\19.png',
   'text': '1. **한국단자 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **미국 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]\n\n3. **멕시코 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]  \n\n*각 항목의 구체적인 수치는 원문에서 확인 필요.*',
   'page': 1,
   'id': 19},
  {'image': 'data/20241121_company_54055000\\23.png',
   'text': '1. **한국단자의 자동차용 ASP 상승률**: 한국단자의 자동차용 ASP는 지속적으로 상승하고 있으며, 최근 몇 년간의 상승률이 두드러짐.\n\n2. **국제 구리 가격 추이 (LME Copper

In [47]:
state_out = create_table_summary_data_batches(state)
state.update(state_out)
state_out

{'table_summary_data_batches': [{'table': 'data/20241121_company_54055000\\1.png',
   'text': '1. **3분기 실적**: 한국단자는 3Q24에서 매출액 3,780억원, 영업이익 485억원을 기록함. 영업이익률은 12.8%로, 전년 대비 +2.5%p 증가.\n\n2. **매출 증가율**: 매출액은 전년 대비 19% 증가, 영업이익은 48% 증가. 영업이익 기준 컨센서스 대비 +30% 서프라이즈.\n\n3. **지역별 매출**:\n   - 한국: 매출액 6% 증가 (YoY)\n   - 폴란드: 매출액 17% 감소 (YoY)\n   - 미국: 매출액 317% 급증 (YoY)\n\n4. **미국/멕시코 법인**: 합산 매출액 비중 21%로, 전년 대비 15%p 상승. 3분기 누적 매출액은 2,283억원으로, 2023년 연간 매출액의 3.3배.\n\n5. **원가 및 수익성**: \n   - 차량용 커넥터 평균가격 5% 상승 (YoY)\n   - 구리 등 비철원재료 원가는 +2.3% 변동, 수지원재료는 -4.4% 변동.\n   - 매출원가율은 -2.6%p (YoY) 감소, 매출총이익률 20.8% 기록.\n\n6. **순이익**: 3분기 세전이익 393억원, 지배주주순이익 338억원으로 각각 16% 및 26% 증가 (YoY).\n\n7. **기대 성장**: 기수주 물량, 신규 고객 발굴, 멕시코 2공장 증설로 높은 성장 기대.\n\n8. **주가 평가**: 현재 P/E 5배대로 저평가 상태, 연간 지배주주순이익 1,400억원 이상 예상.',
   'page': 0,
   'id': 1},
  {'table': 'data/20241121_company_54055000\\13.png',
   'text': '1. **한국단자 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **미국 법인 매출액/순이익 추이**  \n   - 매출액: [

## Image, Table 요약 추출

In [48]:
from langchain_teddynote.models import MultiModal
from langchain_core.runnables import chain


@chain
def extract_image_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """
    당신은 이미지에서 유용한 정보를 추출하는 전문가입니다. 주어진 이미지에서 주요 엔터티를 추출하고, 이를 요약하여 나중에 검색에 사용할 수 있는 유용한 정보를 작성하는 것이 당신의 임무입니다.
    """


    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["image"]
        user_prompt_template = f"""차트와 관련된 내용: {context}
        
###

Output Format:

<image>
<title>
<summary>
<entities> 
</image>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer


@chain
def extract_table_summary(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = """
    당신은 표에서 유용한 정보를 추출하는 전문가입니다. 주어진 이미지에서 주요 엔티티를 추출하고, 이를 요약하여 나중에 검색에 활용할 수 있는 유용한 정보를 작성하는 것이 당신의 임무입니다.
    """

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        context = data_batch["text"]
        image_path = data_batch["table"]
        user_prompt_template = f"""표와 관련된 내용: {context}
        
###

Output Format:

<table>
<title>
<table_summary>
<key_entities> 
<data_insights>
</table>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [49]:
def create_image_summary(state: GraphState):
    # 이미지 요약 추출
    # extract_image_summary 함수를 호출하여 이미지 요약 생성
    image_summaries = extract_image_summary.invoke(
        state["image_summary_data_batches"],
    )

    # 이미지 요약 결과를 저장할 딕셔너리 초기화
    image_summary_output = dict()

    # 각 데이터 배치와 이미지 요약을 순회하며 처리
    for data_batch, image_summary in zip(
        state["image_summary_data_batches"], image_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 이미지 요약 저장
        image_summary_output[data_batch["id"]] = image_summary

    # 이미지 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(image_summary=image_summary_output)


# 이미지 요약 생성 함수 실행
state_out = create_image_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 이미지 요약 결과 출력
state_out["image_summary"]

{16: '<image>\n<title>한국단자 실적 추이</title>\n<summary>이 차트는 2022년부터 2024년까지 한국단자의 매출액과 순이익 추이를 보여줍니다. 매출액은 지속적으로 증가하고 있으며, 순이익도 함께 상승하는 추세입니다. 2024년에는 매출액이 89.3억원, 순이익이 7.2억원에 달할 것으로 예상됩니다.</summary>\n<entities>\n- 한국단자\n- 매출액\n- 순이익\n- 2022년\n- 2023년\n- 2024년\n</entities>\n</image>',
 19: '<image>\n<title>한국단자 실적 추이</title>\n<summary>2022년부터 2024년까지의 한국단자의 매출액과 순이익 추이를 나타낸 차트. 2022년에는 매출액이 1.0십억원, 순이익은 -0.8십억원이었으며, 2024년에는 매출액이 26.4십억원, 순이익이 23.7십억원으로 증가함.</summary>\n<entities>한국단자, 매출액, 순이익, 2022, 2023, 2024</entities>\n</image>',
 23: '<image>\n<title>한국단자의 매출총이익률 추이</title>\n<summary>한국단자의 매출총이익률은 2020년 1.7%에서 2022년 7.0%로 증가하였으며, 2023년에는 4.0%로 소폭 감소하였으나 2024년 3분기에는 5.0%로 회복세를 보일 것으로 예상됨.</summary>\n<entities>\n- 한국단자\n- 매출총이익률\n- 2020년\n- 2021년\n- 2022년\n- 2023년\n- 2024년 3분기\n</entities>\n</image>',
 28: '<image>\n<title>한국단자의 매출총이익률 추이</title>\n<summary>한국단자의 매출총이익률은 2020년 3.9%에서 2021년 19.9%, 2022년 21.4%로 증가하였으며, 2023년에는 11.0%로 감소하였으나 2024년 3분기에는 15.9%로 회복세를 보이고 있다.</summary>\n<entit

In [50]:
def create_table_summary(state: GraphState):
    # 테이블 요약 추출
    table_summaries = extract_table_summary.invoke(
        state["table_summary_data_batches"],
    )

    # 테이블 요약 결과를 저장할 딕셔너리 초기화
    table_summary_output = dict()

    # 각 데이터 배치와 테이블 요약을 순회하며 처리
    for data_batch, table_summary in zip(
        state["table_summary_data_batches"], table_summaries
    ):
        # 데이터 배치의 ID를 키로 사용하여 테이블 요약 저장
        table_summary_output[data_batch["id"]] = table_summary

    # 테이블 요약 결과를 포함한 새로운 GraphState 객체 반환
    return GraphState(table_summary=table_summary_output)


# 테이블 요약 생성 함수 실행
state_out = create_table_summary(state)

# 기존 상태 업데이트
state.update(state_out)

# 테이블 요약 결과 출력
state_out["table_summary"]

{1: '<table>\n<title>3분기 실적 요약</title>\n<key_entities>\n  <entity>한국단자</entity>\n  <entity>3Q24</entity>\n  <entity>매출액</entity>\n  <entity>영업이익</entity>\n  <entity>영업이익률</entity>\n  <entity>지역별 매출</entity>\n  <entity>미국/멕시코 법인</entity>\n  <entity>순이익</entity>\n  <entity>기대 성장</entity>\n  <entity>주가 평가</entity>\n</key_entities>\n<data_insights>\n  <insight>3Q24 매출액 3,780억원, 영업이익 485억원, 영업이익률 12.8%로 전년 대비 증가.</insight>\n  <insight>매출액 19% 증가, 영업이익 48% 증가, 컨센서스 대비 30% 서프라이즈.</insight>\n  <insight>한국 매출 6% 증가, 폴란드 17% 감소, 미국 317% 급증.</insight>\n  <insight>미국/멕시코 법인 매출 비중 21%, 3분기 누적 매출액 2,283억원.</insight>\n  <insight>차량용 커넥터 평균가격 5% 상승, 매출총이익률 20.8% 기록.</insight>\n  <insight>세전이익 393억원, 지배주주순이익 338억원으로 각각 증가.</insight>\n  <insight>기수주 물량 및 신규 고객 발굴로 높은 성장 기대.</insight>\n  <insight>P/E 5배대로 저평가, 연간 지배주주순이익 1,400억원 이상 예상.</insight>\n</data_insights>\n</table>',
 13: '```html\n<table>\n<title>실적 요약</title>\n<table_summary>한국단자 및 법인별 매출액과 순이익 실적 추이</table_summary>\n<key_entities>\n  <entity>한

## Table Markdown 추출

In [51]:
@chain
def table_markdown_extractor(data_batches):
    # 객체 생성
    llm = ChatOpenAI(
        temperature=0,  # 창의성 (0.0 ~ 2.0)
        model_name="gpt-4o-mini",  # 모델명
    )

    system_prompt = "당신은 테이블 이미지의 정보를 마크다운 형식으로 변환하는 전문가입니다. 테이블의 모든 정보를 포함시켜야 하며, 설명은 하지 말고 마크다운 형식으로만 답하십시오"

    image_paths = []
    system_prompts = []
    user_prompts = []

    for data_batch in data_batches:
        image_path = data_batch["table"]
        user_prompt_template = f"""DO NOT wrap your answer in ```markdown``` or any XML tags.
        
###

Output Format:

<table_markdown>

"""
        image_paths.append(image_path)
        system_prompts.append(system_prompt)
        user_prompts.append(user_prompt_template)

    # 멀티모달 객체 생성
    multimodal_llm = MultiModal(llm)

    # 이미지 파일로 부터 질의
    answer = multimodal_llm.batch(
        image_paths, system_prompts, user_prompts, display_image=False
    )
    return answer

In [52]:
def create_table_markdown(state: GraphState):
    # table_markdown_extractor를 사용하여 테이블 마크다운 생성
    # state["table_summary_data_batches"]에 저장된 테이블 데이터를 사용
    table_markdowns = table_markdown_extractor.invoke(
        state["table_summary_data_batches"],
    )

    # 결과를 저장할 딕셔너리 초기화
    table_markdown_output = dict()

    # 각 데이터 배치와 생성된 테이블 마크다운을 매칭하여 저장
    for data_batch, table_summary in zip(
        state["table_summary_data_batches"], table_markdowns
    ):
        # 데이터 배치의 id를 키로 사용하여 테이블 마크다운 저장
        table_markdown_output[data_batch["id"]] = table_summary

    # 새로운 GraphState 객체 반환, table_markdown 키에 결과 저장
    return GraphState(table_markdown=table_markdown_output)


# create_table_markdown 함수 실행
state_out = create_table_markdown(state)

# 기존 state 업데이트
state.update(state_out)

# 생성된 테이블 마크다운 출력
state_out["table_markdown"]

{1: '| **Not Rated**         |                       |\n|-----------------------|-----------------------|\n| **목표주가(12M)**    | Not Rated              |\n| **현재주가(11.20)**  | 70,500원              |\n\n| **Key Data**         |                       |\n|-----------------------|-----------------------|\n| KOSPI 지수 (pt)      | 2,482.29              |\n| 52주 최고가(원)      | 78,200/55,200         |\n| 시가총액(십억원)     | 1,343.4               |\n| 시가총액증(%))       | 0.04                  |\n| 발행주식수(천주)     | 10,415.0              |\n| 60일 평균 거래량(천주)| 37.5                  |\n| 60일 평균 거래대금(십원)| 2.8                  |\n| 외국인 지분율(%)     | 28.18                 |\n| 주요주주 지분율(%)   |                       |\n| - 이창 외 10 인      | 33.19                 |\n| - 국민연금공단       | 10.02                 |\n\n| **Consensus Data**    | 2024      | 2025      |\n|-----------------------|-----------|-----------|\n| 매출액(십억원)       | 1,490     | 1,615     |\n| 영업이익(십억원)     | 168       | 178       |\n| 순이익(십억원)       | 1

## 결과물

In [53]:
state.keys()

dict_keys(['filepath', 'batch_size', 'split_filepaths', 'analyzed_files', 'page_metadata', 'page_elements', 'page_numbers', 'images', 'tables', 'texts', 'text_summary', 'image_summary_data_batches', 'table_summary_data_batches', 'image_summary', 'table_summary', 'table_markdown'])

In [54]:
state["table_summary"]

{1: '<table>\n<title>3분기 실적 요약</title>\n<key_entities>\n  <entity>한국단자</entity>\n  <entity>3Q24</entity>\n  <entity>매출액</entity>\n  <entity>영업이익</entity>\n  <entity>영업이익률</entity>\n  <entity>지역별 매출</entity>\n  <entity>미국/멕시코 법인</entity>\n  <entity>순이익</entity>\n  <entity>기대 성장</entity>\n  <entity>주가 평가</entity>\n</key_entities>\n<data_insights>\n  <insight>3Q24 매출액 3,780억원, 영업이익 485억원, 영업이익률 12.8%로 전년 대비 증가.</insight>\n  <insight>매출액 19% 증가, 영업이익 48% 증가, 컨센서스 대비 30% 서프라이즈.</insight>\n  <insight>한국 매출 6% 증가, 폴란드 17% 감소, 미국 317% 급증.</insight>\n  <insight>미국/멕시코 법인 매출 비중 21%, 3분기 누적 매출액 2,283억원.</insight>\n  <insight>차량용 커넥터 평균가격 5% 상승, 매출총이익률 20.8% 기록.</insight>\n  <insight>세전이익 393억원, 지배주주순이익 338억원으로 각각 증가.</insight>\n  <insight>기수주 물량 및 신규 고객 발굴로 높은 성장 기대.</insight>\n  <insight>P/E 5배대로 저평가, 연간 지배주주순이익 1,400억원 이상 예상.</insight>\n</data_insights>\n</table>',
 13: '```html\n<table>\n<title>실적 요약</title>\n<table_summary>한국단자 및 법인별 매출액과 순이익 실적 추이</table_summary>\n<key_entities>\n  <entity>한

In [55]:
state["table_markdown"]

{1: '| **Not Rated**         |                       |\n|-----------------------|-----------------------|\n| **목표주가(12M)**    | Not Rated              |\n| **현재주가(11.20)**  | 70,500원              |\n\n| **Key Data**         |                       |\n|-----------------------|-----------------------|\n| KOSPI 지수 (pt)      | 2,482.29              |\n| 52주 최고가(원)      | 78,200/55,200         |\n| 시가총액(십억원)     | 1,343.4               |\n| 시가총액증(%))       | 0.04                  |\n| 발행주식수(천주)     | 10,415.0              |\n| 60일 평균 거래량(천주)| 37.5                  |\n| 60일 평균 거래대금(십원)| 2.8                  |\n| 외국인 지분율(%)     | 28.18                 |\n| 주요주주 지분율(%)   |                       |\n| - 이창 외 10 인      | 33.19                 |\n| - 국민연금공단       | 10.02                 |\n\n| **Consensus Data**    | 2024      | 2025      |\n|-----------------------|-----------|-----------|\n| 매출액(십억원)       | 1,490     | 1,615     |\n| 영업이익(십억원)     | 168       | 178       |\n| 순이익(십억원)       | 1

In [56]:
state["image_summary"]

{16: '<image>\n<title>한국단자 실적 추이</title>\n<summary>이 차트는 2022년부터 2024년까지 한국단자의 매출액과 순이익 추이를 보여줍니다. 매출액은 지속적으로 증가하고 있으며, 순이익도 함께 상승하는 추세입니다. 2024년에는 매출액이 89.3억원, 순이익이 7.2억원에 달할 것으로 예상됩니다.</summary>\n<entities>\n- 한국단자\n- 매출액\n- 순이익\n- 2022년\n- 2023년\n- 2024년\n</entities>\n</image>',
 19: '<image>\n<title>한국단자 실적 추이</title>\n<summary>2022년부터 2024년까지의 한국단자의 매출액과 순이익 추이를 나타낸 차트. 2022년에는 매출액이 1.0십억원, 순이익은 -0.8십억원이었으며, 2024년에는 매출액이 26.4십억원, 순이익이 23.7십억원으로 증가함.</summary>\n<entities>한국단자, 매출액, 순이익, 2022, 2023, 2024</entities>\n</image>',
 23: '<image>\n<title>한국단자의 매출총이익률 추이</title>\n<summary>한국단자의 매출총이익률은 2020년 1.7%에서 2022년 7.0%로 증가하였으며, 2023년에는 4.0%로 소폭 감소하였으나 2024년 3분기에는 5.0%로 회복세를 보일 것으로 예상됨.</summary>\n<entities>\n- 한국단자\n- 매출총이익률\n- 2020년\n- 2021년\n- 2022년\n- 2023년\n- 2024년 3분기\n</entities>\n</image>',
 28: '<image>\n<title>한국단자의 매출총이익률 추이</title>\n<summary>한국단자의 매출총이익률은 2020년 3.9%에서 2021년 19.9%, 2022년 21.4%로 증가하였으며, 2023년에는 11.0%로 감소하였으나 2024년 3분기에는 15.9%로 회복세를 보이고 있다.</summary>\n<entit

In [57]:
state["texts"]

{0: '2024년 11월 21일 I 기업분석한국단자 (025540)또 다시 예상보다 좋았어요3Q24 Review: 영업이익률 12.8% 기록한국단자의 3분기 매출액/영업이익은 19%/48% (YoY) 증가한 3,780억원/485억원(영업이\n익률 12.8%, +2.5%p (YoY))으로 2분기에 이어 또 한번 서프라이즈를 기록했다(영업이익\n기준 컨센서스 대비 +30%). 한국(본사/케이이티솔루션/케이티네트워크) 매출액이 6%\n(YoY) 증가했고, 폴란드 매출액은 17% (YoY) 감소했지만 미국 매출액이 317% (YoY) 급\n증했다. 미국/멕시코 법인의 합산 매출액 비중은 21%로 15%p (YoY) 상승했다. 믹스 효\n과로 차량용 커넥터의 평균가격은 3분기 누적 5% (YoY) 상승했다. 구리 등 비철원재료와\n수지원재료(PBT 등)의 투입원가는 누적 기준 각각 +2.3%/-4.4% (YoY) 변동했다. 주요 공\n장의 가동률이 5%p (YoY) 상승하면서 고정비 레버리지 효과가 발생하여 매출원가율이 -\n2.6%p (YoY), +1.2%p (QoQ) 변동했다. 결과로 매출총이익률이 20.8%를 기록했고, 영업\n이익률은 +2.5%p (YoY), -1.7%p (QoQ) 변동한 12.8%로 2분기 14.5%에 이어 2020년\n이후 두 번째로 높은 분기 수익성을 보였다. 3분기가 계절적 비수기임을 고려할 때 3분기\n기준으로는 최고치이다. 영업외 외환손실로 세전이익/지배주주순이익은 영업이익 증가율\n보다 낮은 16%/26% (YoY) 증가한 393억원/338억원을 기록했다.미국 법인의 3분기 누적 매출액이 전년 연간 매출액의 3.3배까지 증가북미 판매/생산을 각각 담당하는 미국/멕시코 법인은 전기차용 ICB(Inter-Connect Board)\n위주로 급성장을 이어가고 있다. 미국/멕시코는 2022년 첫 매출액이 발생한 후 2023년\n689억원/195원을 기록했고, 2024년에는 분기 매출액이 전년 연간 수준을 기록 중이다. 1\n분기 538억원

In [58]:
state["text_summary"]

{0: '1. **3분기 실적**: 한국단자는 3Q24에서 매출액 3,780억원, 영업이익 485억원을 기록함. 영업이익률은 12.8%로, 전년 대비 +2.5%p 증가.\n\n2. **매출 증가율**: 매출액은 전년 대비 19% 증가, 영업이익은 48% 증가. 영업이익 기준 컨센서스 대비 +30% 서프라이즈.\n\n3. **지역별 매출**:\n   - 한국: 매출액 6% 증가 (YoY)\n   - 폴란드: 매출액 17% 감소 (YoY)\n   - 미국: 매출액 317% 급증 (YoY)\n\n4. **미국/멕시코 법인**: 합산 매출액 비중 21%로, 전년 대비 15%p 상승. 3분기 누적 매출액은 2,283억원으로, 2023년 연간 매출액의 3.3배.\n\n5. **원가 및 수익성**: \n   - 차량용 커넥터 평균가격 5% 상승 (YoY)\n   - 구리 등 비철원재료 원가는 +2.3% 변동, 수지원재료는 -4.4% 변동.\n   - 매출원가율은 -2.6%p (YoY) 감소, 매출총이익률 20.8% 기록.\n\n6. **순이익**: 3분기 세전이익 393억원, 지배주주순이익 338억원으로 각각 16% 및 26% 증가 (YoY).\n\n7. **기대 성장**: 기수주 물량, 신규 고객 발굴, 멕시코 2공장 증설로 높은 성장 기대.\n\n8. **주가 평가**: 현재 P/E 5배대로 저평가 상태, 연간 지배주주순이익 1,400억원 이상 예상.',
 1: '1. **한국단자 실적 추이**  \n   - 단위: 십억원, %  \n   - 주요 실적: 매출, 영업이익, 순이익 등\n\n2. **미국 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]\n\n3. **멕시코 법인 매출액/순이익 추이**  \n   - 매출액: [구체적인 수치]  \n   - 순이익: [구체적인 수치]  \n   - 연도별 변화: [구체적인 변화]  \n\n*각 항목의 구체적인 수치는

# Multi_modal_RAG-GTP-4o

In [68]:
!pip install openai
!pip install langchain

In [73]:
!pip install chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached filelock-3.16.1-py3-none-an

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires scikit-learn, which is not installed.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
tensorboard 2.18.0 requires markdown>=2.6.8, which is not installed.
tensorboard 2.18.0 requires werkzeug>=1.0.1, which is not installed.
torch 2.5.0 requires jinja2, which is not installed.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.3 which is incompatible.


In [78]:
# 파일 경로
fpath = "multi-modal/"
fname = "data/20241121_company_54055000.pdf"

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 텍스트 요소의 요약 생성


def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    텍스트 요소 요약
    texts: 문자열 리스트
    tables: 문자열 리스트
    summarize_texts: 텍스트 요약 여부를 결정. True/False
    """

    # 프롬프트 설정
    prompt_text = """당신은 테이블과 텍스트를 요약하여 검색을 위해 사용하는 작업을 맡은 어시스턴트입니다. 이 요약은 임베딩되어 원본 텍스트나 테이블 요소를 검색하는 데 사용됩니다. 검색 최적화가 잘 된 간결한 요약을 제공하십시오. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # 텍스트 요약 체인
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # 요약을 위한 빈 리스트 초기화
    text_summaries = []
    table_summaries = []

    # 제공된 텍스트에 대해 요약이 요청되었을 경우 적용
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # 제공된 테이블에 적용
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# 텍스트, 테이블 요약 가져오기
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)